In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import SGDRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
import datetime

In [3]:
train = pd.read_csv('Hold/trainDfstats.csv') 
test = pd.read_csv('Hold/testDfstats.csv') 

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550068 entries, 0 to 550067
Data columns (total 89 columns):
User_ID                       550068 non-null int64
Product_ID                    550068 non-null object
Gender                        550068 non-null object
Age                           550068 non-null object
Occupation                    550068 non-null int64
City_Category                 550068 non-null object
Stay_In_Current_City_Years    550068 non-null object
Product_Category_1            550068 non-null int64
Product_Category_2            550068 non-null int64
Product_Category_3            550068 non-null int64
Purchase_Amount               550068 non-null int64
Marital_Status                550068 non-null int64
Female                        550068 non-null float64
Male                          550068 non-null float64
Age_0-17                      550068 non-null float64
Age_18-25                     550068 non-null float64
Age_26-35                     550068 non-nul

In [5]:
# Move columns marital status, female, and male to just before binned age and occupation data
# in order to more easily divide data into data frames for testing
cols = list(train)
cols.insert(69, cols.pop(cols.index('Marital_Status')))
cols.insert(69, cols.pop(cols.index('Female')))
cols.insert(69, cols.pop(cols.index('Male')))
trainDf = train.loc[:, cols]

In [6]:
trainDf['Id'] = trainDf['User_ID'].astype(str)+'_'+ trainDf['Product_ID'].astype(str) 
test['Id'] = test['User_ID'].astype(str)+'_'+ test['Product_ID'].astype(str) 

In [7]:
# Creates x and y containing all the custom binned data and all features
yAll = trainDf.iloc[:,10]
xAll = trainDf.iloc[:,11:]

In [8]:
# Converts all categorical data into type int
xAll.iloc[:, :70] = xAll.iloc[:, :70].astype(int)

In [10]:
# Normalizes numeric columns of xAll and creates new dataframe called xAllNorm and yAllNorm
xAllT = xAll.iloc[:, 70:]

cols = [np.arange(70,78)]

xAllNt = xAll.drop(xAll.columns[cols], axis=1)

holdNorm = {}
for column in xAllT:
    
    if column == 'Id':
        temp = xAllT[column]
        temp = temp.reshape(-1, 1)       
        holdNorm[column] = xAllT[column]
        break
        
    scaler = StandardScaler()
    
    temp = xAllT[column].as_matrix()
    temp = temp.reshape(-1,1)
    temp = scaler.fit_transform(temp)
    
    y = temp.flatten()
    holdNorm[column] = y

trainNormDf = pd.DataFrame(holdNorm)
xAllNorm = pd.merge(xAllNt, trainNormDf, on='Id', how='outer')

/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  del sys.path[0]


In [12]:
# Converts all categorical data into type int
yAllNorm = yAll
xAllNorm.iloc[:, :70] = xAllNorm.iloc[:, :70].astype(int)

In [13]:
# Creates a dataframe that only has original binning of data by dropping custom bins but maintains 
# normalized numeric features
cols = xAllNorm.iloc[:, 59:70]
xNbNorm = xAllNorm.drop(cols, axis=1)
yNbNorm = yAllNorm

In [14]:
# Creates a dataframe that only has original binning of data by dropping custom bins and does not 
# have normalized numeric features
cols = xAll.iloc[:, 59:70]
xNb = xAll.drop(cols, axis=1)
yNb = yAll

In [16]:
# Converts all categorical data into type int
xNb.iloc[:, :59] = xNb.iloc[:, :59].astype(int)

In [18]:
xs = [xAll, xAllNorm, xNb, xNbNorm]
ys = [yAll, yAllNorm, yNb, yNbNorm]

In [19]:
# Removes Id column for all data frames
i = 1
for df in xs: 
    print(i)
    df.drop('Id',axis=1, inplace=True)
    i += 1

1
2
3
4


In [23]:
# Prepares test dataframes 
xAllT = test.iloc[:, 10:]
xAllT = xAllT[xAll.columns]

xAllNormT = test.iloc[:, 10:]
xAllNormT = xAllNormT[xAllNorm.columns]

xNbNormT =  test.iloc[:, 10:]
xNbNormT = xNbNormT[xNbNorm.columns]

xNbT = test.iloc[:, 10:]
xNbT = xNbT[xNb.columns]

# Creates user product pair data frame for final submissions
UserProductPairs = test.iloc[:, :2]

In [24]:
testDfs = [xAllT, xAllNormT, xNbNormT, xNbT]

### Linear Regression

In [25]:
# Linear Regression
i = 1
for v in zip(xs, ys):
    x_, y_ = v
    X_train, X_test, y_train, y_test = train_test_split(x_, y_, test_size = .3, random_state=42)
    reg = LinearRegression()
    print('Score for data set:', i)
   
    reg.fit(X_train, y_train)

    y_pred_train = reg.predict(X_train)
    rmseTrain = np.sqrt(mean_squared_error(y_train, y_pred_train))
    print("Train RMSE: {}".format(rmseTrain))

    y_pred = reg.predict(X_test)
    rmseTest = np.sqrt(mean_squared_error(y_test, y_pred))
    print("Test RMSE: {}\n".format(rmseTest))
    
    i += 1

Score for data set: 1
Train RMSE: 4748.60446747989
Test RMSE: 4764.6878065073415

Score for data set: 2
Train RMSE: 4748.637455865274
Test RMSE: 4764.693623573989

Score for data set: 3
Train RMSE: 4748.6044674798895
Test RMSE: 4764.6878065073415

Score for data set: 4
Train RMSE: 4748.619427370744
Test RMSE: 4764.675856257603



In [26]:
# Lasso regression
i = 1
for v in zip(xs, ys):
    x_, y_ = v
    X_train, X_test, y_train, y_test = train_test_split(x_, y_, test_size = .3, random_state=42)
    lasso = Lasso(alpha=.1, normalize=True)
    print('Score for data set:', i)

    lasso.fit(X_train, y_train)

    y_pred_train = lasso.predict(X_train)
    rmseTrain = np.sqrt(mean_squared_error(y_train, y_pred_train))
    print("Train RMSE: {}".format(rmseTrain))

    y_pred = lasso.predict(X_test)
    rmseTest = np.sqrt(mean_squared_error(y_test, y_pred))
    print("Test RMSE: {}\n".format(rmseTest))
    
    i += 1

Score for data set: 1
Train RMSE: 4750.387524741042
Test RMSE: 4766.053731015421

Score for data set: 2
Train RMSE: 4750.387524741986
Test RMSE: 4766.053731016517

Score for data set: 3
Train RMSE: 4750.387524741042
Test RMSE: 4766.053731015421

Score for data set: 4
Train RMSE: 4750.387524741986
Test RMSE: 4766.053731016517



In [27]:
# Gridsearch on lasso regression
param_grid = { "alpha" : [.001, .01, .1, 1, 10, 100, 100]}

i = 1

for v in zip(xs, ys):
    x_, y_ = v
    X_train, X_test, y_train, y_test = train_test_split(x_, y_, test_size = .3, random_state=42)
    lasso = Lasso(normalize=True)
    CV_lasso = GridSearchCV(estimator=lasso, param_grid=param_grid, cv=3)
    CV_lasso.fit(X_train, y_train)
    print('Alpha and Score for data set:', i)

    print('Best Alpha: ', CV_lasso.best_params_)
    bestParams = CV_lasso.best_params_

    lasso = Lasso(alpha = bestParams['alpha'], normalize=True)

    lasso.fit(X_train, y_train)

    y_pred_train = lasso.predict(X_train)
    rmseTrain = np.sqrt(mean_squared_error(y_train, y_pred_train))
    print("Train RMSE: {}".format(rmseTrain))

    y_pred = lasso.predict(X_test)
    rmseTest = np.sqrt(mean_squared_error(y_test, y_pred))
    print("Test RMSE: {}\n".format(rmseTest))
    
    i += 1

Alpha and Score for data set: 1
Best Alpha:  {'alpha': 0.01}
Train RMSE: 4748.7724623315635
Test RMSE: 4764.677345803293

Alpha and Score for data set: 2
Best Alpha:  {'alpha': 0.01}
Train RMSE: 4748.772218169853
Test RMSE: 4764.67678503121

Alpha and Score for data set: 3
Best Alpha:  {'alpha': 0.01}
Train RMSE: 4748.77158354285
Test RMSE: 4764.656496697765

Alpha and Score for data set: 4
Best Alpha:  {'alpha': 0.01}
Train RMSE: 4748.771580407851
Test RMSE: 4764.656480147773



In [28]:
# Ridge regression
i = 1
for v in zip(xs, ys):
    x_, y_ = v
    X_train, X_test, y_train, y_test = train_test_split(x_, y_, test_size = .3, random_state=42)
    ridge = Ridge()
    print('Score for data set:', i)
    
    ridge.fit(X_train, y_train)

    y_pred_train = ridge.predict(X_train)
    rmseTrain = np.sqrt(mean_squared_error(y_train, y_pred_train))
    print("Train RMSE: {}".format(rmseTrain))

    y_pred = ridge.predict(X_test)
    rmseTest = np.sqrt(mean_squared_error(y_test, y_pred))
    print("Test RMSE: {}\n".format(rmseTest))
    
    i += 1

Score for data set: 1
Train RMSE: 4748.604467496582
Test RMSE: 4764.687773711606

Score for data set: 2
Train RMSE: 4748.604467499664
Test RMSE: 4764.687745170561

Score for data set: 3
Train RMSE: 4748.60446750153
Test RMSE: 4764.68778365335

Score for data set: 4
Train RMSE: 4748.604467504582
Test RMSE: 4764.687755112295



In [29]:
# Gridsearch on Ridge regression

param_grid = { "alpha" : [.001, .01, .1, 1, 10, 100, 100]}

i = 1

for v in zip(xs, ys):
    x_, y_ = v
    X_train, X_test, y_train, y_test = train_test_split(x_, y_, test_size = .3, random_state=42)
    ridge = Ridge(normalize=True)
    CV_ridge = GridSearchCV(estimator=ridge, param_grid=param_grid, cv=3)
    CV_ridge.fit(X_train, y_train)
    print('Alpha and Score for data set:', i)

    print('Best Alpha: ', CV_ridge.best_params_)
    bestParams = CV_ridge.best_params_

    ridge = Ridge(alpha = bestParams['alpha'], normalize=True)

    ridge.fit(X_train, y_train)

    y_pred_train = ridge.predict(X_train)
    rmseTrain = np.sqrt(mean_squared_error(y_train, y_pred_train))
    print("Train RMSE: {}".format(rmseTrain))

    y_pred = ridge.predict(X_test)
    rmseTest = np.sqrt(mean_squared_error(y_test, y_pred))
    print("Test RMSE: {}\n".format(rmseTest))
    
    i += 1

Alpha and Score for data set: 1
Best Alpha:  {'alpha': 0.001}
Train RMSE: 4748.6049289849625
Test RMSE: 4764.677292508301

Alpha and Score for data set: 2
Best Alpha:  {'alpha': 0.001}
Train RMSE: 4748.604928984963
Test RMSE: 4764.677292508301

Alpha and Score for data set: 3
Best Alpha:  {'alpha': 0.001}
Train RMSE: 4748.604929295501
Test RMSE: 4764.677352198836

Alpha and Score for data set: 4
Best Alpha:  {'alpha': 0.001}
Train RMSE: 4748.604929295501
Test RMSE: 4764.677352198836



### Random Forest

In [31]:
# Random Forest Regression
print('Starting random forest at: ', datetime.datetime.now())
i = 1
for v in zip(xs, ys):
    x_, y_ = v
    X_train, X_test, y_train, y_test = train_test_split(x_, y_, test_size = .3, random_state=42)
    rfr = RandomForestRegressor(random_state =0)
    print('Score for data set:', i)
   
    rfr.fit(X_train, y_train)

    y_pred_train = rfr.predict(X_train)
    rmseTrain = np.sqrt(mean_squared_error(y_train, y_pred_train))
    print("Train RMSE: {}".format(rmseTrain))

    y_pred = rfr.predict(X_test)
    rmseTest = np.sqrt(mean_squared_error(y_test, y_pred))
    print("Test RMSE: {}\n".format(rmseTest))
    
    i += 1
    
print('Ending random forest at: ', datetime.datetime.now())

Starting random forest at:  2018-04-08 19:51:44.135553
Score for data set: 1
Train RMSE: 2745.245911027414
Test RMSE: 5362.268017925762

Score for data set: 2
Train RMSE: 2746.4819968203237
Test RMSE: 5359.525334597407

Score for data set: 3
Train RMSE: 2746.076319999519
Test RMSE: 5349.703086148489

Score for data set: 4
Train RMSE: 2745.198970816921
Test RMSE: 5352.319362701004

Ending random forest at:  2018-04-08 19:57:31.557924


In [32]:
# Random search on random forest regression
# Saves best parameters in dictionary 
print('Starting at: ', datetime.datetime.now())

param_grid = { 
              "n_estimators": np.arange(200, 500, 50),
              "max_features" : ['auto'],
              "min_samples_leaf": np.arange(10,30,5),
              'max_depth' : np.arange(20, 60, 10)
}

X_train, X_test, y_train, y_test = train_test_split(xs[2], ys[2], test_size = .3, random_state=42)
rfr = RandomForestRegressor(n_jobs=-1, oob_score= True, random_state = 0)
CV_rfr = RandomizedSearchCV(estimator=rfr, param_distributions=param_grid, cv=3)
CV_rfr.fit(X_train, y_train)

print('Best Params: ', CV_rfr.best_params_)
bp = CV_rfr.best_params_ # best parameters

rfr = RandomForestRegressor(
    n_estimators = bp['n_estimators'], 
    max_features=bp['max_features'],
    min_samples_leaf=bp['min_samples_leaf'], 
    max_depth=bp['max_depth'])

rfr.fit(X_train, y_train)



y_pred_train = rfr.predict(X_train)
rmseTrain = np.sqrt(mean_squared_error(y_train, y_pred_train))
print("Train RMSE: {}".format(rmseTrain))

y_pred = rfr.predict(X_test)
rmseTest = np.sqrt(mean_squared_error(y_test, y_pred))
print("Test RMSE: {}\n".format(rmseTest))

print('Ending at: ', datetime.datetime.now())

Starting at:  2018-04-09 13:33:49.394954
Best Params:  {'n_estimators': 450, 'min_samples_leaf': 25, 'max_features': 'auto', 'max_depth': 40}
Train RMSE: 4484.802015812168
Test RMSE: 4805.6168842159195

Ending at:  2018-04-09 16:52:42.985290


In [31]:
# Random search on random forest regression for each data drame
# Saves dataframe and associated best parameters in dictionary 
# for data frame with lowest test RMSE

print('Starting random search on RFR at: ', datetime.datetime.now())

param_grid = { 
              "n_estimators": np.arange(200, 500, 50),
              "max_features" : ['auto'],
              "min_samples_leaf": np.arange(10,30,5),
              'max_depth' : np.arange(20, 60, 10)
}

i = 1

for v in zip(xs, ys):
    
    x_, y_ = v
    
    X_train, X_test, y_train, y_test = train_test_split(x_, y_, test_size = .3, random_state=42)
    
    rfr = RandomForestRegressor(n_jobs=-1, oob_score= True, random_state = 0)
    
    CV_rfr = RandomizedSearchCV(estimator=rfr, param_distributions=param_grid, cv=3)
    
    CV_rfr.fit(X_train, y_train)
    
    print('Parameters and Score for data set:', i)
    print('Best Params: ', CV_rfr.best_params_)
    bp = CV_rfr.best_params_ 

    rfr = RandomForestRegressor(
    n_estimators = bp['n_estimators'], 
    max_features=bp['max_features'],
    min_samples_leaf=bp['min_samples_leaf'], 
    max_depth=bp['max_depth'])

    rfr.fit(X_train, y_train)

    y_pred_train = rfr.predict(X_train)
    rmseTrain = np.sqrt(mean_squared_error(y_train, y_pred_train))
    print("Train RMSE: {}".format(rmseTrain))

    y_pred = rfr.predict(X_test)
    rmseTest = np.sqrt(mean_squared_error(y_test, y_pred))
    print("Test RMSE: {}\n".format(rmseTest))
    
    rmse = rmseTest
    
    if(i == 1):
        best_rmse = rmse
        best_df = i
        bps = bp
    
    if (rmse < best_rmse):
        best_rmse = rmse
        best_df = i
        bps = bp
    
    i +=1
    
print('Ending random search for RFR at: ', datetime.datetime.now())

Starting at:  2018-03-30 07:10:21.667214


ValueError: Invalid parameter max_iter for estimator RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
           oob_score=True, random_state=0, verbose=0, warm_start=False). Check the list of available parameters with `estimator.get_params().keys()`.

In [37]:
bestx = xs[2]
bestxT = testDfs[2]
besty = ys[2]
bp = {'n_estimators': 450, 'min_samples_leaf': 25, 'max_features': 'auto', 'max_depth': 40}

In [ ]:
# Sets best dataframe and best set of parameters for data frame with lowests RMSE
bestx = xs[best_df-1]
bestxT = testDfs[best_df-1
besty = ys[best_df-1]
bp = bps

In [38]:
# Gridsearch on RFR for best dataframe
print('Starting grid search on RFR at: ', datetime.datetime.now())

X_train, X_test, y_train, y_test = train_test_split(bestx, besty, test_size = .3, random_state=42)

rfr = RandomForestRegressor(n_jobs=-1, oob_score= True, random_state = 0) 

param_grid = { 
              "n_estimators": np.arange(200, 500, 50),
              "max_features" : ['auto'],
              "min_samples_leaf": np.arange(10,30,5),
              'max_depth' : np.arange(20, 60, 10)
}

CV_rfr = GridSearchCV(estimator=rfr, param_grid=param_grid, cv= 3)

CV_rfr.fit(X_train, y_train)

bp = CV_rfr.best_params_

print (CV_rfr.best_params_)
print('Ending grid search on RFR at: ', datetime.datetime.now())

Starting grid search on RFR at:  2018-04-10 12:15:35.722759


KeyboardInterrupt: 

In [ ]:
# Feature trimming with Random Forest
print('Starting RFR at: ', datetime.datetime.now())

X_train, X_test, y_train, y_test = train_test_split(bestx, besty, test_size = .3, random_state=42)

rfr = RandomForestRegressor(n_jobs=-1, oob_score= True, random_state = 0, 
                n_estimators = bp['n_estimators'], 
                max_features = bp['max_features'], 
                min_samples_leaf=bp['min_samples_leaf'], 
                max_depth = bp['max_depth'])
    
rfr.fit(X_train, y_train)

y_pred_train = rfr.predict(X_train)
rmseTrain = np.sqrt(mean_squared_error(y_train, y_pred_train))
print("Train RMSE: {}".format(rmseTrain))

y_pred = rfr.predict(X_test)
rmseTest = np.sqrt(mean_squared_error(y_test, y_pred))
print("Test RMSE: {}\n".format(rmseTest))

print('Ending RFR at: ', datetime.datetime.now())

In [ ]:
# Creates a dataframe of feature importances
feature_importance = list(rfr.feature_importances_)
features = list(bestx.columns)


feature_importances = pd.DataFrame({'Feature': features, 'Importance': feature_importance})

print('Feature importance determined from random forest')
print(feature_importances.sort_values('importance', ascending=False))

In [ ]:
# Iterates through subsets of features with increasingly more important 
# subsets of features and calculates the train and test RMSE during each iteration

print('Start random forest feature trimming at: ', datetime.datetime.now())

for i in np.arange(.0008, .0024, .0002):
    threshold = i
    new = features[feature_importance > threshold].tolist()
    newdf = bestx[new]
    print(new)
    
    X_train, X_test, y_train, y_test = train_test_split(newdf, besty, test_size = .1, random_state=42)
    
    rfr.fit(X_train, y_train)

    y_pred_train = rfr.predict(X_train)
    rmseTrain = np.sqrt(mean_squared_error(y_train, y_pred_train))
    print("Train RMSE: {}".format(rmseTrain))

    y_pred = rfr.predict(X_test)
    rmseTest = np.sqrt(mean_squared_error(y_test, y_pred))
    print("Test RMSE: {}\n".format(rmseTest))
    
    rmse = rmseTest
    
    if(i == .0008):
        best_rmse = rmse
        best_df = newdf
        
    
    if (rmse < best_rmse):
        best_rmse = rmse
        best_df = newdf
      
    
    i +=1
    
    
print('Ending RFR trimming at: ', datetime.datetime.now())

In [ ]:
# I was unable to improve the RMSE for random forest regressor from manually trimming 

In [ ]:
# Below produces a submission file for the RFR model

In [ ]:
# Sets RFR with optimized paramters from gridsearch and fits model
rfr = RandomForestRegressor(n_jobs=-1, oob_score= True, random_state = 0, 
                n_estimators = bp['n_estimators'], 
                max_features = bp['max_features'], 
                min_samples_leaf=bp['min_samples_leaf'], 
                max_depth = bp['max_depth'])

rfr.fit(bestx, besty)

In [ ]:
# Produces submission file
y_pred_submission = rfr.predict(bestxT)
y_pred_DF = pd.DataFrame(y_pred_submission)
submission = pd.concat([UP, y_pred_DF], axis=1)
submission = submission1.rename(columns={0: 'Purchase'})
submission.to_csv('Output/rfr.csv')

### Grandient Boosting

In [40]:
# Gradient Boosting Regression
print('Start GBR at: ', datetime.datetime.now())

i = 1
for v in zip(xs, ys):
    x_, y_ = v
    X_train, X_test, y_train, y_test = train_test_split(x_, y_, test_size = .3, random_state=42)
    
    gbr = GradientBoostingRegressor(random_state =0)

    print('Score for data set:', i)

    gbr.fit(X_train, y_train)

    y_pred_train = gbr.predict(X_train)
    rmseTrain = np.sqrt(mean_squared_error(y_train, y_pred_train))
    print("Train RMSE: {}".format(rmseTrain))

    y_pred = gbr.predict(X_test)
    rmseTest = np.sqrt(mean_squared_error(y_test, y_pred))
    print("Test RMSE: {}\n".format(rmseTest))
    
    i += 1
    
print('Ending GBR at: ', datetime.datetime.now())

Start GBR at:  2018-04-10 20:29:47.846803
Score for data set: 1
Train RMSE: 4738.400826743601
Test RMSE: 4758.865574375839

Score for data set: 2
Train RMSE: 4738.400826743601
Test RMSE: 4758.863257942176

Score for data set: 3
Train RMSE: 4738.910670356826
Test RMSE: 4759.622906381364

Score for data set: 4
Train RMSE: 4738.910670356826
Test RMSE: 4759.623073113583

Ending GBR at:  2018-04-10 20:36:37.033911


In [ ]:
# Script to run Randomsearch on each data frame for Gradient Boosting model and then set 
# best paramters based on the randomsearch and rerun for that model 
print('Start random search on GBR at: ', datetime.datetime.now())

param_grid = {'learning_rate':  [.01, .05, 0.15, 0.3, 0.5],
              'max_depth': [8, 9, 10, 11, 12],
              'min_samples_leaf': [ 15, 18, 21, 24],
              'max_features': np.arange(25, 50, 5)
              }  

i = 1

for v in zip(xs, ys):
    x_, y_ = v
    X_train, X_test, y_train, y_test = train_test_split(x_, y_, test_size = .3, random_state=42)
    gbr = GradientBoostingRegressor(random_state =0)
    CV_gbr = RandomizedSearchCV(estimator=gbr, param_distributions=param_grid, cv=3)
    CV_gbr.fit(X_train, y_train)
    print('Best Params and Score for data set:', i, '\n')

    print('Best Params: ', CV_rfr.best_params_ ,'\n')
    bp = CV_gbr.best_params_ # best parameters

    gbr = GradientBoostingRegressor(learning_rate = bp['learning_rate'], 
    max_features=bp['max_features'],
    min_samples_leaf=bp['min_samples_leaf'], 
    max_depth=bp['max_depth'])

    gbr.fit(X_train, y_train)

    y_pred_train = gbr.predict(X_train)
    rmseTrain = np.sqrt(mean_squared_error(y_train, y_pred_train))
    print("Train RMSE: {}".format(rmseTrain))

    y_pred = gbr.predict(X_test)
    rmseTest = np.sqrt(mean_squared_error(y_test, y_pred))
    print("Test RMSE: {}\n".format(rmseTest))
    
    rmse = rmseTest
    
    if(i == 1):
        best_rmse = rmse
        best_df = i
        bps = bp
    
    if (rmse < best_rmse):
        best_rmse = rmse
        best_df = i
        bps = bp
    
    i +=1
    
print('Ending random search for GBR at: ', datetime.datetime.now())

In [ ]:
# Sets best dataframe and best set of parameters for data frame with lowests RMSE
bestx = xs[best_df-1]
bestxT = testDfs[best_df-1
besty = ys[best_df-1]
bp = bps

In [ ]:
# Grid search for GBR
print('Start grid search on GBR at: ', datetime.datetime.now())

X_train, X_test, y_train, y_test = train_test_split(bestx, besty, test_size = .3, random_state=42)
gbr = gbr(n_jobs=-1, oob_score= True, random_state = 0) 


param_grid = {'learning_rate':  [.01, .05, 0.15, 0.3, 0.5],
              'max_depth': [8, 9, 10, 11, 12],
              'min_samples_leaf': [ 15, 18, 21, 24],
              'max_features': np.arange(25, 50, 5)
              }  
print('Starting at: ', datetime.datetime.now())

CV_gbr = GridSearchCV(estimator=gbr, param_grid=param_grid, cv= 3)
CV_gbr.fit(X_train, y_train)
bps = CV_rfc.best_params_
print (CV_rfc.best_params_)

print('Ending grid search on GBR at: ', datetime.datetime.now())

In [ ]:
# Sets gridsearch optimized parameters on GBR model
gbr = GradientBoostingRegressor(learning_rate = bps['learning_rate'], 
    max_features=bps['max_features'],
    min_samples_leaf=bps['min_samples_leaf'], 
    max_depth=bps['max_depth'])

In [ ]:
# Produces submission file for GBR
y_pred_submission = gbr.predict(bestxT)
y_pred_DF = pd.DataFrame(y_pred_submission)
submission = pd.concat([UP, y_pred_DF], axis=1)
submission = submission1.rename(columns={0: 'Purchase'})
submission.to_csv('Output/gbr.csv')

### Neural Network

In [39]:
# Multi-layer perceptron regressor
print('Starting MLPR at: ', datetime.datetime.now())

i = 1
for v in zip(xs, ys):
    x_, y_ = v
    
    X_train, X_test, y_train, y_test = train_test_split(x_, y_, test_size = .3, random_state=42)
    
    mlp = MLPRegressor(random_state =0)

    print('Score for data set:', i)
    
    mlp.fit(X_train, y_train)

    y_pred_train = mlp.predict(X_train)
    rmseTrain = np.sqrt(mean_squared_error(y_train, y_pred_train))
    print("Train RMSE: {}".format(rmseTrain))

    y_pred = mlp.predict(X_test)
    rmseTest = np.sqrt(mean_squared_error(y_test, y_pred))
    print("Test RMSE: {}\n".format(rmseTest))
    
    i += 1
    
print('Ending MLPR at: ', datetime.datetime.now())

Starting MLPR at:  2018-04-10 20:13:05.953755
Score for data set: 1
Train RMSE: 4750.672090536564
Test RMSE: 4766.517437297264

Score for data set: 2


/anaconda/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)


Train RMSE: 4735.771389125665
Test RMSE: 4759.774107837551

Score for data set: 3
Train RMSE: 4749.913677911979
Test RMSE: 4765.9690408197675

Score for data set: 4
Train RMSE: 4738.549853044244
Test RMSE: 4761.111929935567

Ending MLPR at:  2018-04-10 20:29:47.786271


In [ ]:
# Random search on MLPR
print('Starting random search for MLPR at: ', datetime.datetime.now())

param_grid = { 
    "hidden_layer_sizes" : [2, 3],
    "alpha" : [.0001, .001, .01],
    "batch_size" : [1, 10, 50, 100, 250],
    "max_iter"  : np.arange(1250, 2250, 250)
            }

i = 1
for v in zip(xs, ys):
    x_, y_ = v
    
    X_train, X_test, y_train, y_test = train_test_split(x_, y_, test_size = .3, random_state=42)

    mlp = MLPRegressor(random_state = 0) 

    rs_mlp = RandomizedSearchCV(estimator=mlp, param_distributions=param_grid, cv= 3)
    rs_mlp.fit(X_train, y_train)
    print (rs_mlp.best_params_)
    bp = rs_mlp.best_params_

    print('Starting at: ', datetime.datetime.now())

    mlp = MLPRegressor(hidden_layer_sizes = bp['hidden_layer_sizes'], 
    alpha=bp['alpha'],
    batch_size=bp['batch_size'], 
    max_iter=bp['max_iter'])

    mlp.fit(X_train, y_train)

    y_pred_train = mlp.predict(X_train)
    rmseTrain = np.sqrt(mean_squared_error(y_train, y_pred_train))
    print("Train RMSE: {}".format(rmseTrain))

    y_pred = mlp.predict(X_test)
    rmseTest = np.sqrt(mean_squared_error(y_test, y_pred))
    print("Test RMSE: {}\n".format(rmseTest))
    
    rmse = rmseTest
    
    if(i == 1):
        best_rmse = rmse
        best_df = i
        bps = bp
    
    if (rmse < best_rmse):
        best_rmse = rmse
        best_df = i
        bps = bp
    
    i +=1

print('Ending at: ', datetime.datetime.now())
print('Ending random search on MLPR at: ', datetime.datetime.now())

In [ ]:
# Sets best dataframe and best set of parameters for data frame with lowests RMSE
bestx = xs[best_df-1]
bestxT = testDfs[best_df-1
besty = ys[best_df-1]
bp = bps

In [ ]:
# Gridsearch on MLPR
print('Starting gridsearch for MLPR at: ', datetime.datetime.now())

param_grid = { 
    "hidden_layer_sizes" : [2, 3],
    "alpha" : [.0001, .001, .01],
    "batch_size" : [1, 10, 50, 100, 250],
    "max_iter"  : np.arange(1250, 2250, 250)
            }

X_train, X_test, y_train, y_test = train_test_split(xNoNbNorm, yNoNbNorm, test_size = .3, random_state=42)

mlp = MLPRegressor(random_state = 0) 

rs_mlp = GridSearchCV(estimator=mlp, param_grid=param_grid, cv= 3)

rs_mlp.fit(X_train, y_train)

print('Best Params: ', rs_mlp.best_params_)
bp = rs_mlp.best_params_ # best parameters

mlp = MLPRegressor(hidden_layer_sizes = bp['hidden_layer_sizes'], 
alpha=bp['alpha'],
batch_size=bp['batch_size'], 
max_iter=bp['max_iter'])

mlp.fit(X_train, y_train)

y_pred_train = mlp.predict(X_train)
rmseTrain = np.sqrt(mean_squared_error(y_train, y_pred_train))
print("Train RMSE: {}".format(rmseTrain))

y_pred = mlp.predict(X_test)
rmseTest = np.sqrt(mean_squared_error(y_test, y_pred))
print("Test RMSE: {}\n".format(rmseTest))

print('Ending gridsearch for MLPR at: ', datetime.datetime.now())

In [ ]:
# Sets best parameters from grid search on MLPR model
mlp = MLPRegressor(hidden_layer_sizes = bps['hidden_layer_sizes'], 
    alpha=bps['alpha'],
    batch_size=bps['batch_size'], 
    max_iter=bps['max_iter'])

In [ ]:
# Produces MLPR submission
y_pred_submission = mlp.predict(bestxT)
y_pred_DF = pd.DataFrame(y_pred_submission)
submission = pd.concat([UP, y_pred_DF], axis=1)
submission = submission1.rename(columns={0: 'Purchase'})
submission.to_csv('Output/mlpr.csv')